In [68]:
import pandas as pd
from connector import *
import warnings
warnings.filterwarnings("ignore")

In [69]:
conn = create_mysql_engine("creds.json")

In [70]:
q_agg = '''SELECT ffa.name "Name",  fa.fid 'FID', fa.farmer_nid 'NID', ffa.phone "Phone Number",ffbr.fathers_name "Father/Husband",
fbi.bank_name "Bank Name" ,fbi.branch_name "Branch" ,fbi.routing_number "Rounting No." ,fbi.account_number "Account No.",
fa.guarantor_name "Guarantors' Name", fa.guarantor_nid "Guarantors' NID", fa.guarantor_phone "Guarantors' Phone",
fa.project_name "Project Name", ffa.project_duration "Project Duration",fa.loan_amt "Fund Requirement", fa.wegro_service "Wegro ROI (%%)", fa.bank_service "Bank ROI (%%)",fa.input_amt "Input Amount",
fhd.district "Area", fhd.name_en "FO Name" from f_aggrement fa 
left join f_bank_info fbi on fbi.nid = fa.farmer_nid 
left join f_farmers_assesment ffa on ffa.fid = fa.fid 
left join fo_hiararchy_details fhd on fhd.id = ffa.fo_id 
left join f_farmers_bank_requirement ffbr on ffbr.id = ffa.id  '''

In [ ]:
project = 'Maize Harvest -  ভুট্টা'
fo_name = 'Abdul Karim'
area = 'Ullapara'

df = pd.read_sql(q_agg, conn)
df = df[(df['Project Name']==project) & (df['Area']==area) & (df['FO Name']==fo_name)]
df['SL'] = range(1, 1+len(df))

## Summary Table
summary_table = pd.DataFrame(
    {   
        
        'Project': [project.split('-')[0]],
        'Wegro Center Name': [area],
        'FO Name': [fo_name],
        'Total Farmers': [len(df)],
        'Total Fund Requirement': [format(df['Fund Requirement'].sum(),',')],
    }
)


#### Farmer Details

df_farmer = df[['SL',"Name",'FID','NID','Phone Number',"Father/Husband",]]
df_farmer['Name'] = df_farmer['Name'].str.title()
df_farmer['Father/Husband'] = df_farmer['Father/Husband'].str.title()

df_bank = df[['SL','Bank Name','Branch','Rounting No.','Account No.']]


df_guarantor = df[['SL',"Guarantors' Name","Guarantors' NID","Guarantors' Phone"]]
df_guarantor['Guarantors\' Name'] = df_guarantor["Guarantors' Name"].str.title()


df_project = df[['SL','Project Name','Project Duration',"Fund Requirement", 'Input Amount','Wegro ROI (%)','Bank ROI (%)']]
df_project['Project Name'] = df_project['Project Name'].apply(lambda x: x.split(' - ')[0])

In [87]:
summary_table

,Project,Wegro Center Name,FO Name,Total Farmers,Total Fund Requirement
0,Maize Harvest,Ullapara,Abdul Karim,2,"55,435.0"


In [88]:
df_farmer

,SL,Name,FID,NID,Phone Number,Father/Husband
2,1,Md Helel Uddin,F010106783,1453614206,1783413783,Md Usof Ali
4,2,Md Fojlul Haque Mondol,F010103244,2352789503,1723255244,Md Hobibor Rohoman Mondol


In [89]:
df_bank

,SL,Bank Name,Branch,Rounting No.,Account No.
2,1,None,None,None,None
4,2,None,None,None,None


In [90]:
df_guarantor

,SL,Guarantors' Name,Guarantors' NID,Guarantors' Phone
2,1,Erregf,54345,5343534
4,2,Munna Aa,4324352345234,42342345234


In [91]:
df_project

,SL,Project Name,Project Duration,Fund Requirement,Input Amount,Wegro ROI (%),Bank ROI (%)
2,1,Maize Harvest,5,5435.0,4.0,4.0,4.0
4,2,Maize Harvest,5,50000.0,56700.0,4.0,7.0


In [95]:
def df_to_list_of_lists(df):
    return [df.columns.tolist()] + df.values.tolist()